Topic Modelling with Constellate

Topic modelling can be used to extract "topics", from a corpus of texts. The most common algorithm for topic modelling is called Latent Dirichlet Allocation, or LDA. You can use LDA in Constellate, however, because Constellate uses cloud computing, topic modelling with LDA can be very slow. In such cases, it can be quicker to use an algorithm called Non-Negative Matrix Factorization, or NMF. In this tutorial, we will import a corpus, pre-process and tokenize the corpus, and perform topic modelling with LDA and NMF and compare the time they take.

Before doing this tutorial, I recommend that you are familiar with the basics of Constellate and using Python in Jupyter notebooks. It will also help to be familiar with basic text analytics including pre-processing and tokenizing texts, however this code will be supplied in the tutorial. Constellate has many additional tutorials on their website for text analytics.

Both LDA and NMF are available through the Gensim library.

In [3]:
# Import libraries and dataset
import constellate

# If you have created a dataset with the Constellate Dataset Builder, you can input it here
# Otherwise, you may use the dataset supplied with this tutorial

dataset_id = "5e117960-e384-b705-b143-5a667fe614f0"
dataset_file = constellate.get_dataset(dataset_id)

Constellate: use and download of datasets is covered by the Terms & Conditions of Use: https://constellate.org/terms-and-conditions/
All documents from JSTOR published in Past & Present from 1950 - 2020. 2063 documents.
INFO:root:Downloading 5e117960-e384-b705-b143-5a667fe614f0 metadata to /root/data/5e117960-e384-b705-b143-5a667fe614f0-sampled-jsonl.jsonl.gz


100% |########################################################################|


In [7]:
for document in constellate.dataset_reader(dataset_file):
    print("A")

A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A


In [ ]:
# Preprocess and tokenize

In [ ]:
# Stoplist

In [ ]:
# Create dictionary

In [ ]:
# LDA

In [ ]:
# NMF

In [ ]:
# Display topics, qualitative analysis

In [ ]:
# Extract coherence, compare results

Discussion
Text analytic algorithms such as LDA and NMF are often evaluated by quantitative metrics such as perplexity and coherence. However, in the age of Large Language Models, there is also a need to evaluate these algorithms by time and energy usage. Topics must also be evaluated qualitatively by a domain expert. It is my hope that this tutorial has inspired you to think about these algorithms with respect to computational costs and qualitative analysis.

There are other algorithms that can be used for topic modelling, including Biterm Topic Modelling and Correlation Explanation (or CorEx). CorEx differs from other topic modelling algorithms in that it permits the use of "semi-supervised modelling", allowing scholars to provide anchor terms for topics.

Citations

Recommended Reading